Capítulo 6

6.1 Patrones de resumen.

6.1.2. Índice invertido

Patrón que mapea un documento con varias líneas y devuelve la palabra junto a las líneas donde aparece y el número de veces que aparece en cada línea. La clave es la palabra y el valor es una lista con la linea y veces que aparece en cada línea.
Puede ser extrapolable a documentos siempre que estén numerados.

In [1]:
%%writefile indexinverted.py
#!/usr/bin/env python
from mrjob.job import MRJob

class indexinverted(MRJob):
    
    def mapper(self, _, line):
        # Cuidado con el separador
        linea=line.split(' ')
        #Recogemos el primer elemento de la lista "linea", que en este caso identifica el número de línea
        nLinea=linea[0]
        #Recogemos todos los elementos de la lista "linea" a partir de la posición 1 
        palabras=linea[1:]
        #para cada elemento(palabra) de la lista "valor" lo pasamos al Reducer en minúscula y con su número de línea. 
        for palabra in palabras:
            yield palabra.lower(),nLinea
    
    #La característica especial del reducer, es que se va a encargar de hacer un tratamiento por cada palabra que encuentre
    #Lo que vamos a controlar es el número de línea y veces que aparece una palabra en una línea, 
    #mediante una lista. El resto se encarga el Reducer. 
    def reducer(self, key, values):
        listaValores=[]
        listaDef=[]
        #Introducimos en una lista los valores que vienen del mapper
        for valor in values:
            listaValores.append(valor)
        
        #Vamos a ir viendo en cada elemento de la lista que es lo que contiene,
        #para ello usamos el contador "posicion" que nos permitirá recorrer y extraer
        #cada elemento de la lista mediante su número de posición. 
        
        for posicion in range(len(listaValores)):
            # la subLista contendrá el número de línea/documento donde aparece la palabra y 
            # el número de veces que aparece.
            subLista=[]
            
            #Metemos el número de línea/documento en la variable elementoLista
            elementoLista=listaValores[posicion]
            
            #Contamos el número de elementos iguales que hay en listaValores
            #Metemos el número de elementos en la variable numérica cuentaElementos
            cuentaElementos=listaValores.count(elementoLista) 
            
            #Con las dos variables anteriores construimos el elemento de la sublista
            subLista=[listaValores[posicion],cuentaElementos]
            
            #Si esa sublista creada está en la lista definitiva, no la agregamos por que ya ha sido agregada anteriormente
            #si no, lo agregamos a la lista definitiva.
            if subLista not in listaDef:
                listaDef.append(subLista)
                
        #Mandamos a la salida la palabra (key) y las líneas/documentos(listaDef), y número
        #de veces que aparecen
        yield key, listaDef
    
if __name__ == '__main__':
    indexinverted.run()


Writing indexinverted.py


In [2]:
! python indexinverted.py archivos_datos/docejemplo.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/indexinverted.root.20201214.183816.522876
Running step 1 of 1...
job output is in /tmp/indexinverted.root.20201214.183816.522876/output
Streaming final output from /tmp/indexinverted.root.20201214.183816.522876/output...
"es"	[["linea4", 1]]
"mismo"	[["linea4", 1]]
"nada"	[["linea3", 1]]
"perro"	[["linea1", 1], ["linea2", 1]]
"que"	[["linea2", 1]]
"rapidamente"	[["linea3", 1]]
"rapido"	[["linea1", 1]]
"y"	[["linea3", 1]]
"de"	[["linea4", 1]]
"el"	[["linea1", 1], ["linea2", 2], ["linea3", 1], ["linea4", 3]]
"camino"	[["linea4", 1]]
"chavea"	[["linea2", 1], ["linea3", 1]]
"chico"	[["linea4", 1]]
"con"	[["linea2", 1]]
"corre"	[["linea1", 1], ["linea2", 2], ["linea3", 1]]
Removing temp directory /tmp/indexinverted.root.20201214.183816.522876...


EJECUCIÓN EN EL CLUSTER:

In [3]:
! python indexinverted.py hdfs:///archivos_datos/docejemplo.txt -r hadoop --python-bin /opt/anaconda/bin/python

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/lib/hadoop/bin...
Found hadoop binary: /usr/lib/hadoop/bin/hadoop
Using Hadoop version 2.6.0
Looking for Hadoop streaming jar in /usr/lib/hadoop...
Looking for Hadoop streaming jar in /usr/lib/hadoop-mapreduce...
Found Hadoop streaming jar: /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
Creating temp directory /tmp/indexinverted.root.20201214.183954.387404
uploading working dir files to hdfs:///user/root/tmp/mrjob/indexinverted.root.20201214.183954.387404/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/indexinverted.root.20201214.183954.387404/files/
Running step 1 of 1...
  packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7934080213853834604.jar tmpDir=null
  Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
  Connecting to ResourceManager at yarnmaster/172.18.0.5:8032
  Total input 